In [ ]:
%reload_ext autoreload
%autoreload 2

import time

start = time.time()
from torch_snippets import *
end = time.time()
print(end-start)

3.5508437156677246

In [ ]:
from torch_snippets.misc import Timer
import time

n = 5
x = Timer(n)
for i in range(n):
    time.sleep(0.2)
    x()

In [ ]:
%%writefile tmp.txt
a
batchify
callable
dcopy

Writing tmp.txt


In [ ]:
P('tmp.txt').rm('you want to remove {self}? [y/N]')

you want to remove tmp.txt? [y/N]y


In [ ]:
from torch_snippets.ipython import backup_this_notebook

backup_this_notebook('scripts.ipynb', '/tmp/scripts')

<IPython.core.display.Javascript object>

[03/05/23 00:24:48] INFO     Saved the notebook at scripts.ipynb!                       ]8;id=85162;file:///Users/yeshwanth.y/code/torch_snippets/torch_snippets/ipython.py\ipython.py]8;;\:]8;id=535778;file:///Users/yeshwanth.y/code/torch_snippets/torch_snippets/ipython.py#save_notebook:37\save_notebook:37]8;;\

                    INFO     Backed a new version of notebook at                 ]8;id=11285;file:///Users/yeshwanth.y/code/torch_snippets/torch_snippets/ipython.py\ipython.py]8;;\:]8;id=612420;file:///Users/yeshwanth.y/code/torch_snippets/torch_snippets/ipython.py#backup_this_notebook:55\backup_this_notebook:55]8;;\
                             /tmp/scripts/0000.html                                                                